In [1]:
import os 

In [2]:
%pwd

'e:\\ML_Projects_iNeuron\\iNeuron_Project_Census_data_Classification_With_MLflow\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'e:\\ML_Projects_iNeuron\\iNeuron_Project_Census_data_Classification_With_MLflow'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    # alpha: float
    # l1_ratio: float
    target_column: str

In [6]:
from Mlflow_Ineuron_Project.constants import *
from Mlflow_Ineuron_Project.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            # alpha = params.alpha,
            # l1_ratio = params.l1_ratio,
            target_column = schema.name
            
        )

        return model_trainer_config

In [ ]:
import pandas as pd
import os
from Mlflow_Ineuron_Project import logger
# from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

import joblib

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        classification_model=DecisionTreeClassifier()
        classification_model.fit(train_x,train_y)
        y_pred=classification_model.predict(test_x)
        report=classification_report(test_y,y_pred)
        print(report)

        joblib.dump(classification_model, os.path.join(self.config.root_dir, self.config.model_name))



In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-04-23 11:06:17,155: INFO: common: yaml file: config\config.yaml loaded successfully]


[2025-04-23 11:06:17,179: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-23 11:06:17,201: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-04-23 11:06:17,201: INFO: common: created directory at: artifacts]
[2025-04-23 11:06:17,201: INFO: common: created directory at: artifacts/model_trainer]
              precision    recall  f1-score   support

           0       0.89      0.87      0.88      6192
           1       0.61      0.64      0.63      1949

    accuracy                           0.82      8141
   macro avg       0.75      0.76      0.75      8141
weighted avg       0.82      0.82      0.82      8141

